<a href="https://colab.research.google.com/github/MaSSI89/MINI-COMPILER/blob/main/recognize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import LIbraries

DataProcessing


In [9]:
# Imports
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torchvision import models
from torchvision.transforms import transforms
import pickle
import copy
import time
import kagglehub

dataset_path = kagglehub.dataset_download("gulerosman/hg14-handgesture14-dataset")
print("Path to dataset:", dataset_path)



Path to dataset: /kaggle/input/hg14-handgesture14-dataset


NameError: name 'data_processor' is not defined

In [18]:
class DataProcessor:
    def __init__(self, data_path=None, img_size=128):
        self.data_path = data_path
        self.img_size = img_size
        self.X_train = None
        self.y_train = None
        self.X_val = None
        self.y_val = None
        self.X_test = None
        self.y_test = None

    def load_hg14_dataset(self, download=False):
        import os
        import cv2
        import numpy as np
        import torch
        from sklearn.model_selection import train_test_split

        if download:
            try:
              !kaggle datasets download -d gulerosman/hg14-handgesture14-dataset -p /root/.cache/kagglehub/datasets/gulerosman/hg14-handgesture14-dataset/versions/1 --unzip
              print("Dataset downloaded successfully.")
            except:
                print("Please download the HG14 dataset manually from: ")
                print("https://www.kaggle.com/datasets/gulerosman/hg14-handgesture14-dataset")
                print("and place it in the 'data' directory")
                return False

        if self.data_path is None:
            self.data_path = "D:\\m1\\Rnn\\projet\\HG14-Hand Gesture"

        if not os.path.exists(self.data_path):
            print(f"Dataset not found at {self.data_path}")
            return False

        print("Loading dataset...")

        class_folders = [f for f in os.listdir(self.data_path)
                        if os.path.isdir(os.path.join(self.data_path, f))]

        images = []
        labels = []

        for class_idx, class_name in enumerate(class_folders):
            class_path = os.path.join(self.data_path, class_name)
            print(f"Processing class {class_name} ({class_idx})...")

            image_files = [f for f in os.listdir(class_path)
                          if f.endswith(('.jpg', '.jpeg', '.png'))]

            for img_file in image_files:
                img_path = os.path.join(class_path, img_file)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (self.img_size, self.img_size))
                    images.append(img)
                    labels.append(class_idx)

        X = np.array(images)
        y = np.array(labels)

        X_train_val, X_test, y_train_val, y_test = train_test_split(
            X, y, test_size=0.1, random_state=42, stratify=y)

        X_train, X_val, y_train, y_val = train_test_split(
            X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val)

        X_train = X_train / 255.0
        X_val = X_val / 255.0
        X_test = X_test / 255.0

        num_classes = len(np.unique(y))

        self.X_train = torch.FloatTensor(X_train.transpose(0, 3, 1, 2))
        self.X_val = torch.FloatTensor(X_val.transpose(0, 3, 1, 2))
        self.X_test = torch.FloatTensor(X_test.transpose(0, 3, 1, 2))

        self.y_train = torch.LongTensor(y_train)
        self.y_val = torch.LongTensor(y_val)
        self.y_test = torch.LongTensor(y_test)

        self.y_train_onehot = torch.nn.functional.one_hot(self.y_train, num_classes).float()
        self.y_val_onehot = torch.nn.functional.one_hot(self.y_val, num_classes).float()
        self.y_test_onehot = torch.nn.functional.one_hot(self.y_test, num_classes).float()

        print(f"Dataset processed. {len(X_train)} training, {len(X_val)} validation, {len(X_test)} test samples.")
        return True

    def collect_custom_dataset(self, num_classes=14, samples_per_class=100):
        import os
        import cv2
        import time

        if not os.path.exists('data/custom_dataset'):
            os.makedirs('data/custom_dataset')

        cap = cv2.VideoCapture(0)

        for class_idx in range(num_classes):
            class_dir = f'data/custom_dataset/class_{class_idx}'
            os.makedirs(class_dir, exist_ok=True)

            print(f"Collecting images for gesture class {class_idx}")
            print("Position your hand in the frame and press 's' to start capture")
            print("Press 'q' to quit")

            wait_key = True
            count = 0

            while wait_key:
                ret, frame = cap.read()
                if not ret:
                    break

                cv2.putText(frame, f"Class: {class_idx}, Samples: {count}/{samples_per_class}",
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(frame, "Press 's' to start/pause capture",
                           (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                cv2.imshow('Capture', frame)
                key = cv2.waitKey(1) & 0xFF

                if key == ord('s'):
                    wait_key = False
                elif key == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    return False

            print("Capturing... (Press 'p' to pause, 'q' to quit)")

            while count < samples_per_class:
                ret, frame = cap.read()
                if not ret:
                    break

                h, w = frame.shape[:2]
                cv2.rectangle(frame, (w//4, h//4), (3*w//4, 3*h//4), (0, 255, 0), 2)

                cv2.putText(frame, f"Class: {class_idx}, Samples: {count}/{samples_per_class}",
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                cv2.imshow('Capture', frame)
                key = cv2.waitKey(1) & 0xFF

                if key == ord('p'):
                    print("Paused. Press 's' to resume")
                    wait_pause = True
                    while wait_pause:
                        key = cv2.waitKey(1) & 0xFF
                        if key == ord('s'):
                            wait_pause = False
                        elif key == ord('q'):
                            cap.release()
                            cv2.destroyAllWindows()
                            return False
                elif key == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    return False

                roi = frame[h//4:3*h//4, w//4:3*w//4]
                roi = cv2.resize(roi, (self.img_size, self.img_size))
                cv2.imwrite(f'{class_dir}/sample_{count}.jpg', roi)
                count += 1
                time.sleep(0.1)

            print(f"Completed capturing for class {class_idx}")

        cap.release()
        cv2.destroyAllWindows()
        print("Dataset collection complete.")

        self.data_path = 'data/custom_dataset'
        return self.load_custom_dataset()

    def load_custom_dataset(self):
        import os
        import cv2
        import numpy as np
        import torch
        from sklearn.model_selection import train_test_split

        if not os.path.exists(self.data_path):
            print(f"Custom dataset not found at {self.data_path}")
            return False

        class_folders = [f for f in os.listdir(self.data_path)
                        if os.path.isdir(os.path.join(self.data_path, f))]

        images = []
        labels = []

        for class_idx, class_name in enumerate(sorted(class_folders)):
            class_path = os.path.join(self.data_path, class_name)
            image_files = [f for f in os.listdir(class_path)
                          if f.endswith(('.jpg', '.jpeg', '.png'))]

            for img_file in image_files:
                img_path = os.path.join(class_path, img_file)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (self.img_size, self.img_size))
                    images.append(img)
                    labels.append(class_idx)

        X = np.array(images)
        y = np.array(labels)

        X_train_val, X_test, y_train_val, y_test = train_test_split(
            X, y, test_size=0.1, random_state=42, stratify=y)

        X_train, X_val, y_train, y_val = train_test_split(
            X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val)

        X_train = X_train / 255.0
        X_val = X_val / 255.0
        X_test = X_test / 255.0

        num_classes = len(np.unique(y))

        self.X_train = torch.FloatTensor(X_train.transpose(0, 3, 1, 2))
        self.X_val = torch.FloatTensor(X_val.transpose(0, 3, 1, 2))
        self.X_test = torch.FloatTensor(X_test.transpose(0, 3, 1, 2))

        self.y_train = torch.LongTensor(y_train)
        self.y_val = torch.LongTensor(y_val)
        self.y_test = torch.LongTensor(y_test)

        self.y_train_onehot = torch.nn.functional.one_hot(self.y_train, num_classes).float()
        self.y_val_onehot = torch.nn.functional.one_hot(self.y_val, num_classes).float()
        self.y_test_onehot = torch.nn.functional.one_hot(self.y_test, num_classes).float()

        print(f"Custom dataset processed. {len(X_train)} training, {len(X_val)} validation, {len(X_test)} test samples.")
        return True

    def load_gesture_dataset(self):
        import os
        import cv2
        import numpy as np
        import torch
        from sklearn.model_selection import train_test_split

        if not os.path.exists(self.data_path):
            print(f"Dataset not found at {self.data_path}")
            return False

        print("Loading gesture dataset...")

        print(os.listdir(self.data_path))
        for item in os.listdir(self.data_path):
            print(item)

        class_folders = [f for f in os.listdir(self.data_path)
                        if os.path.isdir(os.path.join(self.data_path, f)) and f.startswith('Gesture_')]

        class_folders.sort()

        images = []
        labels = []

        for class_folder in class_folders:
            class_idx = int(class_folder.split('_')[1])
            class_path = os.path.join(self.data_path, class_folder)
            print(f"Processing {class_folder} (class {class_idx})...")

            image_files = [f for f in os.listdir(class_path)
                        if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

            for img_file in image_files:
                img_path = os.path.join(class_path, img_file)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, (self.img_size, self.img_size))
                    images.append(img)
                    labels.append(class_idx)

        X = np.array(images)
        y = np.array(labels)

        X_train_val, X_test, y_train_val, y_test = train_test_split(
            X, y, test_size=0.1, random_state=42, stratify=y)

        X_train, X_val, y_train, y_val = train_test_split(
            X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val)

        X_train = X_train / 255.0
        X_val = X_val / 255.0
        X_test = X_test / 255.0

        self.X_train = torch.FloatTensor(X_train.transpose(0, 3, 1, 2))
        self.X_val = torch.FloatTensor(X_val.transpose(0, 3, 1, 2))
        self.X_test = torch.FloatTensor(X_test.transpose(0, 3, 1, 2))

        num_classes = 14

        self.y_train = torch.LongTensor(y_train)
        self.y_val = torch.LongTensor(y_val)
        self.y_test = torch.LongTensor(y_test)

        self.y_train_onehot = torch.nn.functional.one_hot(self.y_train, num_classes).float()
        self.y_val_onehot = torch.nn.functional.one_hot(self.y_val, num_classes).float()
        self.y_test_onehot = torch.nn.functional.one_hot(self.y_test, num_classes).float()

        print(f"Dataset processed. {len(X_train)} training, {len(X_val)} validation, {len(X_test)} test samples.")
        return True

    def get_data(self):
        return self.X_train, self.y_train_onehot, self.X_val, self.y_val_onehot, self.X_test, self.y_test_onehot

    def save_data(self, filename='data/processed_data.pkl'):
        import pickle
        import os

        os.makedirs(os.path.dirname(filename), exist_ok=True)

        data = {
            'X_train': self.X_train,
            'y_train': self.y_train_onehot,
            'X_val': self.X_val,
            'y_val': self.y_val_onehot,
            'X_test': self.X_test,
            'y_test': self.y_test_onehot
        }
        with open(filename, 'wb') as f:
            pickle.dump(data, f)
        print(f"Data saved to {filename}")

    def load_data(self, filename='data/processed_data.pkl'):
        import pickle
        import os

        if not os.path.exists(filename):
            print(f"Data file {filename} not found")
            return False

        with open(filename, 'rb') as f:
            data = pickle.load(f)

        self.X_train = data['X_train']
        self.y_train_onehot = data['y_train']
        self.X_val = data['X_val']
        self.y_val_onehot = data['y_val']
        self.X_test = data['X_test']
        self.y_test_onehot = data['y_test']

        print(f"Data loaded from {filename}")
        return True

## Gesture Recognition Model

The `GestureRecognitionModel` class defines the CNN and transfer learning models, training, evaluation, and prediction methods.
CODE SELL GESTURERECOGNITION MODEL CLASS

In [15]:
class GestureRecognitionModel:
    def __init__(self, input_shape=(3, 128, 128), num_classes=14):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = None
        self.history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    def build_custom_cnn(self):
        model = nn.Sequential(
            nn.Conv2d(self.input_shape[0], 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, self.num_classes)
        )
        self.model = model.to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
        return model

    def build_mobilenet(self):
        base_model = models.mobilenet_v2(weights='IMAGENET1K_V1')
        for param in base_model.parameters():
            param.requires_grad = False
        num_features = base_model.classifier[1].in_features
        base_model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_features, self.num_classes)
        )
        self.model = base_model.to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
        return self.model

    def build_vgg16(self):
        base_model = models.vgg16(weights='IMAGENET1K_V1')
        for param in base_model.features.parameters():
            param.requires_grad = False
        base_model.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, self.num_classes)
        )
        self.model = base_model.to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
        return self.model

    def build_ensemble_model(self, model_paths):
        models_list = []
        for path in model_paths:
            model = torch.load(path)
            model.eval()
            models_list.append(model.to(self.device))
        self.ensemble_models = models_list
        return models_list

    def train(self, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
        if self.model is None:
            print("Model not built. Please build a model first.")
            return
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataset = TensorDataset(X_val, y_val)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)
        best_val_loss = float('inf')
        patience = 10
        patience_counter = 0
        best_model_weights = None
        os.makedirs('models', exist_ok=True)
        for epoch in range(epochs):
            self.model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            for inputs, labels in train_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                if labels.shape[1] > 1:
                    labels = torch.argmax(labels, dim=1)
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            train_loss = running_loss / len(train_loader.dataset)
            train_acc = correct / total
            self.model.eval()
            val_loss = 0.0
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs = inputs.to(self.device)
                    labels = labels.to(self.device)
                    if labels.shape[1] > 1:
                        labels = torch.argmax(labels, dim=1)
                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, labels)
                    val_loss += loss.item() * inputs.size(0)
                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_loss = val_loss / len(val_loader.dataset)
            val_acc = correct / total
            print(f'Epoch {epoch+1}/{epochs} - '
                  f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
                  f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                best_model_weights = copy.deepcopy(self.model.state_dict())
                torch.save(self.model, 'models/best_model.pth')
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f'Early stopping triggered after {epoch+1} epochs')
                    break
        if best_model_weights is not None:
            self.model.load_state_dict(best_model_weights)
        return self.history

    def evaluate(self, X_test, y_test, batch_size=32):
        if self.model is None:
            print("Model not built. Please build a model first.")
            return
        test_dataset = TensorDataset(X_test, y_test)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)
        self.model.eval()
        test_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                if labels.shape[1] > 1:
                    labels = torch.argmax(labels, dim=1)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                test_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        test_loss = test_loss / len(test_loader.dataset)
        test_acc = correct / total
        print(f"Test loss: {test_loss:.4f}")
        print(f"Test accuracy: {test_acc:.4f}")
        return [test_loss, test_acc]

    def predict(self, X):
        if self.model is None:
            print("Model not built. Please build a model first.")
            return None
        if not isinstance(X, torch.Tensor):
            X = torch.FloatTensor(X)
        X = X.to(self.device)
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(X)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
        return probabilities.cpu().numpy()

    def predict_ensemble(self, X):
        if not hasattr(self, 'ensemble_models') or len(self.ensemble_models) == 0:
            print("Ensemble models not loaded. Please build ensemble models first.")
            return None
        if not isinstance(X, torch.Tensor):
            X = torch.FloatTensor(X)
        X = X.to(self.device)
        predictions = []
        for model in self.ensemble_models:
            model.eval()
            with torch.no_grad():
                outputs = model(X)
                probabilities = torch.nn.functional.softmax(outputs, dim=1)
                predictions.append(probabilities.cpu().numpy())
        ensemble_pred = np.mean(predictions, axis=0)
        return ensemble_pred

    def save_model(self, filepath='models/gesture_model.pth'):
        if self.model is None:
            print("Model not built. Please build a model first.")
            return
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        torch.save(self.model, filepath)
        print(f"Model saved to {filepath}")

    def load_model(self, filepath='models/gesture_model.pth'):
        if not os.path.exists(filepath):
            print(f"Model file {filepath} not found")
            return False
        self.model = torch.load(filepath)
        self.model.to(self.device)
        print(f"Model loaded from {filepath}")
        return True

    def plot_training_history(self):
        if not self.history['train_loss']:
            print("No training history available.")
            return
        os.makedirs('logs', exist_ok=True)
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(self.history['train_acc'])
        plt.plot(self.history['val_acc'])
        plt.title('Model Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.subplot(1, 2, 2)
        plt.plot(self.history['train_loss'])
        plt.plot(self.history['val_loss'])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.tight_layout()
        plt.savefig('logs/training_history.png')
        plt.show()

## Training Workflow

The following function orchestrates the data loading, model building, training, evaluation, and saving.
CODE CELL , TRAIN GESTURE RECOGNITION MODEL FUNCTION

In [16]:
def train_gesture_recognition_model():
    # 1. Initialize the data processor with path to your dataset

    data_processor = DataProcessor(data_path="/root/.cache/kagglehub/datasets/gulerosman/hg14-handgesture14-dataset/versions/1", img_size=128)
    print(os.listdir(data_processor.data_path))

    # 2. Load and process the gesture dataset
    success = data_processor.load_gesture_dataset()
    if not success:
        print("Failed to load dataset")
        return

    # 3. Get the processed data
    X_train, y_train, X_val, y_val, X_test, y_test = data_processor.get_data()

    # 4. Save the processed data (optional)
    data_processor.save_data('data/gesture_dataset.pkl')

    # 5. Initialize the model for 14 gesture classes
    model = GestureRecognitionModel(input_shape=(3, 128, 128), num_classes=14)

    # 6. Build the model (you can choose from different architectures)
    # Option 1: Custom CNN
    model.build_custom_cnn()
    # Option 2: MobileNet transfer learning
    # model.build_mobilenet()
    # Option 3: VGG16 transfer learning
    # model.build_vgg16()

    # 7. Train the model
    history = model.train(X_train, y_train, X_val, y_val, epochs=50, batch_size=32)

    # 8. Evaluate on test set
    test_results = model.evaluate(X_test, y_test)

    # 9. Save the trained model
    model.save_model('models/gesture_recognition_model.pth')

    # 10. Plot training history
    model.plot_training_history()

    return model

Run TRAINING:
Execute the training workflow

In [20]:
if __name__ == "__main__":
    train_gesture_recognition_model()


['HG14']
Loading gesture dataset...


ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.